In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [3]:
#Reading CO2 data from file
CO2_raw_data = pd.read_csv("..\\resources\\CO2.csv")

In [4]:
#Deleting undesired columns from CO2_data_raw
CO2_data = CO2_raw_data.drop(columns=["Country Code", "Indicator Name", "Indicator Code"]).iloc[:,:-5]

In [5]:
#Deleting rows with NaN values from CO2_data
CO2_data.dropna(axis=0, how="any", inplace=True)
CO2_data.reset_index(inplace=True, drop=True)

In [6]:
#Reading country temperatures raw data
temp_country_raw = pd.read_csv("..\\resources\\GlobalLandTemperaturesByCountry.csv")

In [7]:
#Removing undesired column from data and converting dt row to datetime format
temp_country = temp_country_raw.drop(columns=["AverageTemperatureUncertainty"])
temp_country["dt"] = pd.to_datetime(temp_country["dt"])

In [8]:
#Getting list of countries from CO2_data
country_list_CO2 = list(CO2_data["Country Name"])

In [10]:
#Deleting temperatures from countries without CO2 data
temp_country_clean = temp_country[temp_country["Country"].isin(country_list_CO2)]

In [11]:
#Getting list of countries from temp_country_clean
country_list_temp = list(temp_country_clean["Country"].unique())

In [107]:
#Deleting countries without temp data
CO2_data_clean = CO2_data[CO2_data["Country Name"].isin(country_list_temp)]

#Reformating data frame to have Countries by columns
columns = list(CO2_data_clean["Country Name"])
CO2_data_clean = CO2_data_clean.transpose()
CO2_data_clean.columns = columns
CO2_data_clean.drop(index="Country Name", inplace = True)
CO2_data_clean["Year"] = CO2_data_clean.index

#Reordering columns
CO2_data_clean = CO2_data_clean[["Year"]+columns]

#Changing 'Year' column type to datetime
CO2_data_clean["Year"] = pd.to_datetime(CO2_data_clean["Year"])

#Resetting dataframe indexes
CO2_data_clean.set_index("Year", inplace=True)
CO2_data_clean.sort_index(axis=1, inplace=True)

#Now CO2_data_clean is ready to work with it
CO2_data_clean.head(2)

,Afghanistan,Albania,Algeria,Angola,Argentina,Aruba,Australia,Austria,Bahrain,Bangladesh,...,Togo,Tonga,Tunisia,Turkey,Uganda,United Arab Emirates,United Kingdom,United States,Uruguay,Vietnam
Year,,,,,,,,,,,,,,,,,,,,,
1960-01-01,0.0460567,1.25819,0.55712,0.100835,2.38334,204.62,8.58294,4.37332,3.54448,0.294805,...,0.0417624,0.178654,0.41337,0.612271,0.062317,0.119035,11.1508,15.9998,1.70159,0.181947
1961-01-01,0.0535888,1.37419,0.535024,0.0822038,2.45855,208.823,8.64157,4.49636,10.5493,0.296924,...,0.0573856,0.172659,0.417045,0.616879,0.0582829,0.109141,11.1541,15.6813,1.60273,0.183099


In [46]:
#Getting temperatures from 1910 (50 years before CO2 data) to 2013 (there is no data after that date)
temp_country_clean = temp_country_clean.loc[(temp_country_clean["dt"] > "1910-01-01") & (temp_country_clean["dt"] < "2013-09-01")]

#Are there any NaN values in that slice?
temp_country_clean["AverageTemperature"].isna().sum()

6

In [47]:
#Checking The countries with incomplete data
for country in country_list_temp:
    b = temp_country_clean.loc[a["Country"] == country]
    if b["AverageTemperature"].isna().sum() != 0:
        print(country, "-", b["AverageTemperature"].isna().sum())

French Polynesia - 6


In [70]:
#Reformating temperature data
aux = temp_country_clean.loc[temp_country_clean["Country"] == country_list_temp[0]]
aux.rename(columns={"dt":"Date","AverageTemperature":country_list_temp[0]}, inplace=True)
aux.drop(columns="Country", inplace = True)

#Changing 'Year' column type to datetime
aux["Date"] = pd.to_datetime(aux["Date"])

#Resetting dataframe indexes
aux.set_index("Date", inplace=True)

In [76]:
#Adding all countries
for i in range(len(country_list_temp)):
    aux[country_list_temp[i]] = list(temp_country_clean.loc[temp_country_clean["Country"] == country_list_temp[i]]["AverageTemperature"])
temp_country_final = aux

In [108]:
#Now temp_country_final is ready to work with it
temp_country_final.head(2)

,Afghanistan,Albania,Algeria,Angola,Argentina,Aruba,Australia,Austria,Bahrain,Bangladesh,...,Togo,Tonga,Tunisia,Turkey,Uganda,United Arab Emirates,United Kingdom,United States,Uruguay,Vietnam
Date,,,,,,,,,,,,,,,,,,,,,
1910-02-01,3.851,7.284,13.918,22.766,19.077,25.260,27.435,-0.608,17.303,19.763,...,28.534,25.841,10.705,3.695,23.313,20.208,4.780,-4.243,21.489,19.921
1910-03-01,6.727,6.722,17.787,22.736,16.474,25.239,24.410,2.077,18.589,24.372,...,28.719,25.374,13.804,4.112,23.395,22.258,5.998,5.972,18.844,21.841


In [112]:
resampled = temp_country_final.resample("Y").mean()
resampled.head(2)

,Afghanistan,Albania,Algeria,Angola,Argentina,Aruba,Australia,Austria,Bahrain,Bangladesh,...,Togo,Tonga,Tunisia,Turkey,Uganda,United Arab Emirates,United Kingdom,United States,Uruguay,Vietnam
Date,,,,,,,,,,,,,,,,,,,,,
1910-12-31,14.391,13.431182,23.721364,21.503364,13.597091,27.691545,21.000182,7.065364,26.167091,25.206364,...,26.424455,22.876818,20.262000,12.43900,22.551818,27.925818,8.840091,9.941364,15.787909,23.843000
1911-12-31,13.385,12.483583,22.574667,21.224333,13.886333,27.580500,21.347917,6.732333,24.717083,24.503500,...,25.992000,22.774667,19.780583,10.51575,23.103667,26.847083,9.068917,8.737917,15.885250,23.796167


In [109]:
len(list(resampled.columns))

124

In [118]:
#Reading coord data from file
coord_raw = pd.read_csv("..\\resources\\coord.csv")

#Dropping columns
coord_raw.drop(columns="country", inplace=True)

#Renaming columns
coord_raw.rename(columns={"latitude":"Lat", "longitude":"Lon", "name":"Country"}, inplace=True)
coord_raw

,Lat,Lon,Country
0,42.546245,1.601554,Andorra
1,23.424076,53.847818,United Arab Emirates
2,33.939110,67.709953,Afghanistan
3,17.060816,-61.796428,Antigua and Barbuda
4,18.220554,-63.068615,Anguilla
...,...,...,...
240,15.552727,48.516388,Yemen
241,-12.827500,45.166244,Mayotte
242,-30.559482,22.937506,South Africa
243,-13.133897,27.849332,Zambia


In [124]:
coord_clean = coord_raw.loc[coord_raw["Country"].isin(list(resampled.columns))]
coord_clean = coord_clean.set_index("Country")
coord_clean = coord_clean.T
coord_clean.sort_index(axis=1, inplace=True)
coord_clean

Country,Afghanistan,Albania,Algeria,Angola,Argentina,Aruba,Australia,Austria,Bahrain,Bangladesh,...,Togo,Tonga,Tunisia,Turkey,Uganda,United Arab Emirates,United Kingdom,United States,Uruguay,Vietnam
Lat,33.939110,41.153332,28.033886,-11.202692,-38.416097,12.521110,-25.274398,47.516231,25.930414,23.684994,...,8.619543,-21.178986,33.886917,38.963745,1.373333,23.424076,55.378051,37.090240,-32.522779,14.058324
Lon,67.709953,20.168331,1.659626,17.873887,-63.616672,-69.968338,133.775136,14.550072,50.637772,90.356331,...,0.824782,-175.198242,9.537499,35.243322,32.290275,53.847818,-3.435973,-95.712891,-55.765835,108.277199


In [136]:
# libraries
from mpl_toolkits.basemap import Basemap
 
# Always start witht the basemap function to initialize a map
m=Basemap()
 
# Then add element: draw coast line, map boundary, and fill continents:
m.drawcoastlines()
m.drawmapboundary()
m.fillcontinents()
 
# You can add rivers as well
#m.drawrivers(color='#0000ff')
 
plt.show()

ModuleNotFoundError: No module named 'mpl_toolkits.basemap'